<a href="https://colab.research.google.com/github/oyhenart/IAO-Football-Analytics/blob/main/notebooks/Mapas_de_pases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1️⃣ Preparación de datos


## 1.1 Instalar e Importar librerías


In [ ]:
# Instalar las librerias
!pip install matplotlib
!pip install urllib
!pip install mplsoccer
!pip install PIL
!pip install statsbombpy

In [ ]:
#Importar lo necesario
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgba
from mplsoccer import Pitch, FontManager, VerticalPitch, add_image
from urllib.request import urlopen
from PIL import Image
from statsbombpy import sb

## 1.2 Limpiar datos
- Filtrar columnas necesarias
- Revisar valores nulos o inconsistentes

In [ ]:
partido = sb.events(match_id=3869685)

In [ ]:
partido.columns

In [ ]:
partido[['x','y']] = partido['location'].apply(pd.Series)
partido[['end_x','end_y']] = partido['pass_end_location'].apply(pd.Series)

In [ ]:
partido.pass_outcome.value_counts()
pases_partido = partido[(partido['type'] == 'Pass') & (partido['team'] == 'Argentina')]
pases_partido.shape

In [ ]:
pases_partido.pass_outcome.value_counts()
completados = pases_partido[pases_partido['pass_outcome'].isna() == True]

## 1.3 Exploración inicial
- Estadísticas básicas
- Conteo de pases por jugador

In [ ]:
completados[['player', 'location', 'pass_end_location']]

In [ ]:
completados[['player','x','y','end_x','end_y']]

In [ ]:
completados.groupby('player')[['x','y']].mean()

In [ ]:
completados['pasador'] = completados['player']
completados['receptor'] = completados['player'].shift(-1)

In [ ]:
completados[['pasador','receptor']]

In [ ]:
firstSub = partido[(partido['type'] == 'Substitution') & (partido['team'] == 'Argentina')].minute.min()
firstSub

In [ ]:
completados= completados[completados['minute']< firstSub]

In [ ]:
average_locations = completados.groupby('player').agg({'x': ['mean'], 'y': ['mean', 'count']})
average_locations.columns = ['x', 'y', 'count']
average_locations

In [ ]:
pass_between = completados.groupby(['pasador', 'receptor']).id.count().reset_index()
pass_between.rename({'id': 'pass_count'}, axis= 'columns', inplace=True)

In [ ]:
pass_between = pass_between.merge(average_locations, left_on='pasador', right_index= True)
pass_between = pass_between.merge(average_locations, left_on='receptor', right_index= True, suffixes=['','_end'])
pass_between

In [ ]:
pass_between = pass_between[pass_between['pass_count']> 1]
pass_between

In [ ]:
MAX_LINE_WIDTH = 13
pass_between['width'] = (pass_between.pass_count / pass_between.pass_count.max() * MAX_LINE_WIDTH)

MIN_TRANSPARENCY = 0.000001
color=np.array(to_rgba('yellow'))
color=np.tile(color, (len(pass_between),1))
c_transparency= pass_between.pass_count / pass_between.pass_count.max()
c_transparency= (c_transparency * (1 - MIN_TRANSPARENCY)) + MIN_TRANSPARENCY
color[:, 3] = c_transparency

MAX_MARKER_SIZE=1200
average_locations['marker_size'] = (average_locations['count'] / average_locations['count'].max() * MAX_MARKER_SIZE)

# 2️⃣ Visualización de pases


# 2️⃣ Visualización de pases
## 2.1 Crear campo de fútbol
- Usar `mplsoccer` para dibujar el campo


In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
pitch = VerticalPitch()

pitch.draw(ax=ax)

pitch.scatter(50,70, ax=ax)
pitch.scatter(40,80, ax=ax)

## 2.2 Dibujar pases con flechas
- Dirección y longitud de los pases
- Color, grosor y transparencia para claridad

## 2.3 Ajustes estéticos
- Leyendas, títulos y etiquetas
- Personalización por jugador o tipo de pase


In [ ]:
pitch = VerticalPitch(pitch_type= 'statsbomb', pitch_color='#22312b', line_color='#c7d5cc', goal_type='box', goal_alpha=.8)
fig, ax = pitch.draw(figsize=(16, 11), constrained_layout=False, tight_layout=True)
fig.set_facecolor('#22312b')

arrows = pitch.arrows(pass_between.x, pass_between.y, pass_between.x_end, pass_between.y_end, ax=ax, color=color, zorder=.99)
nodes= pitch.scatter(average_locations.x, average_locations.y, ax=ax, color='white', ec='black', s=average_locations['marker_size'])

for index, row in pass_between[['pasador', 'x', 'y']].drop_duplicates(subset='pasador').iterrows():
  pitch.annotate(row.pasador, xy=(row.x-4.5, row.y), c='white', va='center',
                  ha='center', size=14, weight='bold', ax=ax, zorder=1)

# 3️⃣ Conclusiones
A partir del análisis de los pases en el partido:

- ⚡ Se observan patrones de pase predominantes en ciertas zonas del campo.
- 🔄 Algunos jugadores realizan combinaciones cortas consecutivas.
- 🎯 Los pases largos se concentran en momentos de transición rápida al ataque.
- 🛡️ Se identifican zonas de pérdida de balón, posibles áreas de mejora defensiva.
- 📊 El mapa de flechas permite visualizar dirección, longitud y frecuencia de los pases.
